<a href="https://colab.research.google.com/github/DaniDeutsch/CMSC320_HW1_Summer2024/blob/main/CMSC320_HW1_Part_2_SQL_DD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1 Part 2: SQL
# Due Date: June 4th, 2024, by 11:59 PM
### *Late Due Date with a 5% LATE PENALTY: June 5th, 2024, by 11:59 PM*
To start with, if you felt the class was unclear, check out the following tutorial: https://mode.com/sql-tutorial/introduction-to-sql/

Now! We'll be using sqlite to access a database. Start by downloading the sql lite file and putting it in the same directory as this notebook: https://www.kaggle.com/datasets/kaggle/sf-salaries (hit the 'download' button in the upper right).

### DO NOT REMOVE ANY PART OF ANY OF THE QUESTIONS OR YOU LOSE CREDIT
### REMEMBER TO SHOW ALL CODE OUTPUT (NO CREDIT OTHERWISE)

### Before we proceed, please note that every task **must be completed using a single SQL query**, unless mentioned otherwise or given as prompt.

#### (e.g. using print statements are fine, but you should not be using **pandas library** to work with the dataset.).

You can use one "Print" statement with the single SQL questions.

The following code will use sqlite to create a database connection.
#### Run the following code first:

In [34]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("database.sqlite")
crsr = conn.cursor()

Check out the description of the data so you know the table / column names.

In [35]:
query = "SELECT name FROM sqlite_master WHERE type='table';"

tables = crsr.execute(query).fetchall()
print(tables)

[('Salaries',), ('Y2011',), ('Y2012',), ('Y2013',), ('Y2014',)]


# Section 01: Exploration (TOTAL POINTS: 2)
## **COMPLETE EACH TASK WITH ONE SINGLE SQL QUERY**

**1.1. (2 POINTS)**


Try to create a query that gives you a data frame of the **EmployeeName, JobTitle, and BasePay** from the salaries table.

**LIMIT** the table to the **year 2012**, **employees** having a **base pay under 150,000**, and **sort in descending order** by **base pay**.

In [36]:
# ANSWER BY ONLY FILLING OUT THE QUERY (You can remove this comment)
query = 'SELECT EmployeeName, JobTitle, BasePay FROM salaries WHERE BasePay < 150000 AND Year = 2012 ORDER BY BasePay DESC'

df = pd.read_sql(query, conn)
df.head()

,EmployeeName,JobTitle,BasePay
0,Julie Kirschbaum,Project Manager 3,149881.06
1,Eric Jamison,Senior Physician Specialist,149746.72
2,Daisy Mae Quitoriano,Nurse Manager,149657.87
3,Ruth Armstrong,Nurse Practitioner,149169.48
4,Stuart Washington,Senior Physician Specialist,148811.73


# Section 02: Aggregation  (TOTAL POINTS: 5)
## **COMPLETE EACH TASK WITH ONE SINGLE SQL QUERY**


**2.1. (1 POINT)**

Produce and print the head of a dataframe that shows the average pay for each year (only use a single, simple query). Your result should have a column for the year and a column for the average base pay.

In [37]:
# ANSWER BY ONLY FILLING OUT THE QUERY (You can remove this comment)
query = 'SELECT Year, AVG(BasePay) AS AverageBasePay FROM salaries GROUP BY Year'

df = pd.read_sql(query, conn)
df.head()

,Year,AverageBasePay
0,2011,63595.956517
1,2012,65436.406857
2,2013,68509.832156
3,2014,66557.437750


**2.2. (4 POINTS)**

Create a dataframe with averages of base pay, averages of benefits, and averages of overtime for each job title, as well as a column with the sum of these three values.

In [38]:
# ANSWER BY ONLY FILLING OUT THE QUERY (You can remove this comment)
query = """
        SELECT JobTitle, AVG(BasePay), AVG(Benefits), AVG(OvertimePay), (AVG(BasePay) + AVG(Benefits) + AVG(OvertimePay)) AS TotalPay
        FROM salaries
        GROUP BY JobTitle
"""

df = pd.read_sql(query, conn)
df.head()

,JobTitle,AVG(BasePay),AVG(Benefits),AVG(OvertimePay),TotalPay
0,ACCOUNT CLERK,43300.806506,0.00,373.200843,43674.007349
1,ACCOUNTANT,46643.172000,0.00,0.000000,46643.172000
2,ACCOUNTANT INTERN,28732.663958,0.00,24.430625,28757.094583
3,"ACPO,JuvP, Juv Prob (SFERS)",62290.780000,17975.59,0.000000,80266.370000
4,ACUPUNCTURIST,66374.400000,0.00,0.000000,66374.400000


 # Section 03: Table Creation  (TOTAL POINTS: 3)
## **COMPLETE EACH TASK WITH ONE SINGLE SQL QUERY**
**3.1. (3 POINTS)**

Now we'll create our own table in our database. Separate the Salaries table by Year, and add it back to the database. (You may use basic python to complete the task, however, should still use SQL to query the data.)


To do this, you'll have to learn how to create a table in SQL. You will likely need this function (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html), which should make the process simple.

Here's the basic structure of how you'll want to do this...

1. Through some form of loop, collect all the salaries in a given year.
2. Write that data to the sql connection using the function linked above. Make sure to give it a unique name!

In [39]:
for y in ['2011','2012','2013','2014']:
# ANSWER BY ONLY FILLING OUT THE QUERY (You can remove this comment)
    query = f"SELECT * FROM salaries WHERE Year = {y}"

    df = pd.read_sql(query, conn)
    df.to_sql(name='Y'+y, con=conn, if_exists='replace')

**do not modify below code** (ONLY FOR DEBUGGING PURPOSE FOR 3.1.)

*You can run the code below to check that things work. Your output should be similar to the expected output below... but the names of your new tables do not need to match exactly*.

In [40]:
### it is added to check if db has the tables added in above problem 8 ###
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

## if your above code is correct, in that case, running this cell will create output like following
## [('Salaries',), ('2011',), ('test',), ('Y2011',), ('Y2012',), ('Y2013',), ('Y2014',)]

[('Salaries',), ('Y2011',), ('Y2012',), ('Y2013',), ('Y2014',)]


# Section 04: Table Joining  (TOTAL POINTS: 3)
## **COMPLETE EACH TASK WITH ONE SINGLE SQL QUERY**

We'll move on to a new dataset for the next steps.
Download the dataset from here (https://www.kaggle.com/datasets/luizpaulodeoliveira/imdb-project-sql) and load the sqlite file same as before.

**4.1. (1 POINT)** Start by just selecting everything in the "movies" table to see what it looks like.

In [41]:
conn = sqlite3.connect("movies.sqlite")
# ANSWER BY ONLY FILLING OUT THE QUERY (You can remove this comment)
query = 'SELECT * FROM movies'

df = pd.read_sql(query, conn)
df.head()

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766


**4.2. (2 POINTS)**

Create a dataframe that includes the entire contents of "movies" table as well as the director's name.

**Hint:** You may want to perform an inner join on the 'directors' table to complete this task. Investigate that table to learn more about this solution. Note that the movies table has a 'director_id' column which may help.

In [42]:
# ANSWER BY ONLY FILLING OUT THE QUERY (You can remove this comment)
query = 'SELECT movies.*, directors.name AS director_name FROM movies INNER JOIN directors ON movies.director_id = directors.id'

df = pd.read_sql(query, conn)
print(df.head(5))
df.size

      id                            original_title     budget  popularity  \
0  43597                                    Avatar  237000000         150   
1  43598  Pirates of the Caribbean: At World's End  300000000         139   
2  43599                                   Spectre  245000000         107   
3  43600                     The Dark Knight Rises  250000000         112   
4  43601                               John Carter  260000000          43   

  release_date     revenue                                     title  \
0   2009-12-10  2787965087                                    Avatar   
1   2007-05-19   961000000  Pirates of the Caribbean: At World's End   
2   2015-10-26   880674609                                   Spectre   
3   2012-07-16  1084939099                     The Dark Knight Rises   
4   2012-03-07   284139100                               John Carter   

   vote_average  vote_count  \
0           7.2       11800   
1           6.9        4500   
2          

66822

# Section 05: Analysis  (TOTAL POINTS: 7)
## **COMPLETE EACH TASK WITH ONE SINGLE SQL QUERY**

The next few problems will be more involved! You'll need to combine some concepts you've learned.

**For each cell, fix the query.**

**5.1. (3 POINTS)**

Which directors have the **highest** overall voting average? - show the **top 5 directors' name** and their average rating

In [43]:
query = 'SELECT directors.name as NameOfDirector, AVG(movies.vote_average) as AvgRating '
# FIX THE LINES OF THE SINGLE SQL QUERY (Fix when prompted. There is only one thing to be changed).

# START HERE
query += 'FROM directors ' # Fix, Don't get rid of the space at the end

query += 'INNER JOIN movies ' # Fix, Don't get rid of the space at the end

query += 'ON directors.id = movies.director_id ' # Don't Fix

query += 'GROUP BY director_id '  # Fix, Don't get rid of the space at the end

query += 'ORDER BY AvgRating DESC LIMIT 5;' # Don't Fix

df = pd.read_sql(query, conn)
print(df)

  NameOfDirector  AvgRating
0    Gary Sinyor      10.00
1   Rohit Jugraj       9.50
2     Lance Hool       9.30
3   Floyd Mutrux       8.50
4  Tim McCanlies       8.45


**5.2. (4 POINTS)**

What are the **top five directors** by *how much* their **average budget** is?

In [44]:
query = 'SELECT budget, d.name AS director_name '
# FIX THE LINES OF THE SINGLE SQL QUERY (Fix when prompted. There is only one thing to be changed).

# START HERE
query += 'FROM movies as m ' # Fix, Don't get rid of the space at the end

query += 'INNER JOIN directors AS d ' # Don't Fix

query += 'ON d.id = m.director_id ' # Fix, Don't get rid of the space at the end

query += 'GROUP BY d.name ' # Fix, Don't get rid of the space at the end

query += 'ORDER BY avg(budget) DESC LIMIT 5;' # Fix

df = pd.read_sql(query, conn)
print(df)

      budget   director_name
0  260000000    Byron Howard
1  200000000     Lee Unkrich
2  200000000     Dan Scanlon
3  250000000     David Yates
4  185000000  Brenda Chapman
